In [16]:
import boto3

In [17]:
s3 = boto3.client('s3')

In [18]:
if 'tdf-tht' not in [n['Name'] for n in s3.list_buckets()['Buckets']]:
    s3.create_bucket(
                ACL='public-read',
                Bucket='tdf-tht',
                CreateBucketConfiguration={'LocationConstraint': 'ap-southeast-2'}
            )
    print('The new bucket (tdf-tht) has been created.')
else :
    print('The bucket (tdf-tht) exists, please proceeding further work.')

The bucket (tdf-tht) exists, please proceeding further work.


In [19]:
try:
    s3.get_object(Bucket='tdf-tht', Key='Staging/weatherDataHourly.csv')
except :
    head = 'weather,temp,temp_min,temp_max,pressure,humidity,visibility,datetime,sunrise,sunset'
    s3.put_object(Body=head, Bucket='tdf-tht', Key='Staging/weatherDataHourly.csv')

In [21]:
s3.get_object(Bucket='tdf-tht', Key='Staging/weatherDataHourly.csv')

{'ResponseMetadata': {'RequestId': 'TV8YECG253Q86EYS',
  'HostId': 'qpw2skqrzcOx6pZX+3rFuKV/4dRi+uVTWmf1ex4QyYPf9ibdfG5MsFDv9GuJepL7RDNCNwo+P7E=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'qpw2skqrzcOx6pZX+3rFuKV/4dRi+uVTWmf1ex4QyYPf9ibdfG5MsFDv9GuJepL7RDNCNwo+P7E=',
   'x-amz-request-id': 'TV8YECG253Q86EYS',
   'date': 'Mon, 20 Dec 2021 02:33:19 GMT',
   'last-modified': 'Mon, 20 Dec 2021 02:07:06 GMT',
   'etag': '"ee08d298d08f53ad68a7eeba9d8f9dd2"',
   'accept-ranges': 'bytes',
   'content-type': 'binary/octet-stream',
   'server': 'AmazonS3',
   'content-length': '183'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2021, 12, 20, 2, 7, 6, tzinfo=tzutc()),
 'ContentLength': 183,
 'ETag': '"ee08d298d08f53ad68a7eeba9d8f9dd2"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f8e7196dcd0>}

In [25]:
import requests
cityName = 'Sydney'
APIKey = 
response = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q={cityName}&units=metric&appid={APIKey}").json()
print(response)

{'coord': {'lon': 151.2073, 'lat': -33.8679}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 28.4, 'feels_like': 31.31, 'temp_min': 25.76, 'temp_max': 32.01, 'pressure': 1009, 'humidity': 69}, 'visibility': 10000, 'wind': {'speed': 1.34, 'deg': 127, 'gust': 3.13}, 'clouds': {'all': 40}, 'dt': 1639967667, 'sys': {'type': 2, 'id': 2018875, 'country': 'AU', 'sunrise': 1639939224, 'sunset': 1639991087}, 'timezone': 39600, 'id': 2147714, 'name': 'Sydney', 'cod': 200}


In [26]:
{'weather':response['weather'][0]['main'], 
           'temp':response['main']['temp'], 
           'temp_min':response['main']['temp_min'], 
           'temp_max':response['main']['temp_max'],
           'pressure':response['main']['pressure'], 
           'humidity':response['main']['humidity'],
           'visibility':response['visibility'],
           'datetime':datetime.fromtimestamp(response['dt']),
           'sunrise':datetime.fromtimestamp(response['sys']['sunrise']), 
           'sunset':datetime.fromtimestamp(response['sys']['sunset'])}

{'weather': 'Clouds',
 'temp': 28.4,
 'temp_min': 25.76,
 'temp_max': 32.01,
 'pressure': 1009,
 'humidity': 69,
 'visibility': 10000,
 'datetime': datetime.datetime(2021, 12, 20, 13, 34, 27),
 'sunrise': datetime.datetime(2021, 12, 20, 5, 40, 24),
 'sunset': datetime.datetime(2021, 12, 20, 20, 4, 47)}

In [28]:
import pandas as pd
from datetime import datetime
import os
s3.download_file('tdf-tht', 'Staging/weatherDataHourly.csv','file_name')
df = pd.read_csv('file_name')
new_row = {'weather':response['weather'][0]['main'], 
           'temp':response['main']['temp'], 
           'temp_min':response['main']['temp_min'], 
           'temp_max':response['main']['temp_max'],
           'pressure':response['main']['pressure'], 
           'humidity':response['main']['humidity'],
           'visibility':response['visibility'],
           'datetime':datetime.fromtimestamp(response['dt']),
           'sunrise':datetime.fromtimestamp(response['sys']['sunrise']), 
           'sunset':datetime.fromtimestamp(response['sys']['sunset'])}
df = df.append(new_row, ignore_index=True)
bytes_to_write = df.to_csv(None, index=False).encode()
s3.put_object(Body=bytes_to_write, Bucket='tdf-tht', Key='Staging/weatherDataHourly.csv')
os.remove('file_name')

In [29]:
s3.download_file('tdf-tht', 'Staging/weatherDataHourly.csv','file_name')
df = pd.read_csv('file_name')
df

,weather,temp,temp_min,temp_max,pressure,humidity,visibility,datetime,sunrise,sunset
0,Clouds,28.18,25.76,31.14,1009,69,10000,2021-12-20 12:46:27,2021-12-20 05:40:24,2021-12-20 20:04:47
1,Clouds,28.40,25.76,32.01,1009,69,10000,2021-12-20 13:34:27,2021-12-20 05:40:24,2021-12-20 20:04:47
